<a href="https://colab.research.google.com/github/antoinebachand/Deep-Learnig-/blob/main/Lab_Antoine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab-Exam**

In [1]:
# Import necessary libraries.
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms

torch.manual_seed(42)  # Do not change.


In [18]:
# NOTE: Do not change anyting in this code cell.
# Utility functions for training and evaluating your networks.

def train(model, dl, num_epocs, opt, criterion, input_cv):
  for epochs in range(num_epocs):
    for data in train_dl:
      X, y = data
      output = model(input_cv(X))
      loss = criterion(output, y)
      optimizer.zero_grad() 
      loss.backward() 
      optimizer.step() 
    print('Loss:', loss.data)


def evaluate(model, dl, input_cv):
  total, correct = 0, 0
  with torch.no_grad():
    for data in dl:
      X, y = data
      output = model(input_cv(X))
      for idx, val in enumerate(output):
        if torch.argmax(val) == y[idx]:
          correct += 1
        total += 1
  return round(correct/total, 3)


**Question 1**: In the code cell below, write data loaders for the train- and test-datasets, using the specified batch_size. Make sure to shuffle the training data.

In [3]:
# QUESTION 1

MNIST_DIM = 28 # The x, y demensions of the MNIST images. 
MNIST_NUM_CLASSES = 10 # Number of classes.

# Download the MNIST dataset.
train_ds = datasets.MNIST('',train=True, download=True,
                          transform=transforms.Compose([transforms.ToTensor()]))
test_ds = datasets.MNIST('',train=False, download=True,
                          transform=transforms.Compose([transforms.ToTensor()]))

# Create dataloaders from the train- and test-sets with a given batch size, 
# called test_dl and test_dl, respectively.

BATCH_SIZE = 200

train_dl = torch.utils.data.DataLoader(train_ds, batch_size=200, shuffle=True)
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=200, shuffle=False)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting MNIST/raw/train-images-idx3-ubyte.gz to MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting MNIST/raw/train-labels-idx1-ubyte.gz to MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST/raw



**Question 2**: In the code cell below,  create a fully connected neural network, called Q2Net, that takes the digit images as input and classifies them as one of the digits from 0-9. The specification of your network are as follows:
* The network has two hidden layers, the former of width *256* and the latter of with *128*. They should use ReLU actication.
* There should be a *batch normalization* layer following the first hidden layer.
* The output layer is fully-connected of with *64* using the most appropriate activation for the task. 
* You should give the input and output sizes (for the problem) as parameters to the net's constructor.




In [19]:


class Q2Net(nn.Module):
  
  def __init__(self, input_size, output_size): 
    super().__init__()
    self.fc1 = nn.Linear(28*28, 64)  # I know the dimension of the images is 16 x 16.
    self.av1 = nn.ReLU()
    self.fc2 = nn.Linear(64, 64)
    self.fc2_bn = nn.BatchNorm1d(64)  
    self.av2 = nn.ReLU()
    self.fc3 = nn.Linear(64, 64)
    self.av3 = nn.ReLU()
    self.fcout = nn.Linear(64, 10)  # I know there is 10 possible output
    return
    


  def forward(self, x):
    x = self.fc1(x)
    x = self.av1(x)
    x = self.fc2(x)
    x = self.fc2_bn(x)
    x = self.av2(x)
    x = self.fc3(x)
    x = self.av3(x)
    x = self.fcout(x)
    return nn.functional.log_softmax(x, dim=1)
    

net = Q2Net(MNIST_DIM*MNIST_DIM, MNIST_NUM_CLASSES)
print(net)


Q2Net(
  (fc1): Linear(in_features=784, out_features=64, bias=True)
  (av1): ReLU()
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc2_bn): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (av2): ReLU()
  (fc3): Linear(in_features=64, out_features=64, bias=True)
  (av3): ReLU()
  (fcout): Linear(in_features=64, out_features=10, bias=True)
)


**Question 3**: Use the provided *training(...) function to train your network using the
the specified *learning rate* and number of *epochs*.  Use *RMSprop* as an optimizer and the most suitable *loss function* for the task (and your network).

In [20]:
# Train the neural network using the provided parameters and using
# RMSprop as an optimizer. 

LEARNING_RATE = 0.001
NUM_EPOCHS = 5
optimizer = torch.optim.RMSprop(net.parameters(), lr=0.001)
loss_func = nn.CrossEntropyLoss()

# This function you use to covert the input record from the dataset
# into the format expected by your network 
def input_converter(X):
  return X.view(-1, MNIST_DIM*MNIST_DIM)

train(net, train_dl, NUM_EPOCHS, optimizer, loss_func, input_converter)


Loss: tensor(0.1199)
Loss: tensor(0.1170)
Loss: tensor(0.0936)
Loss: tensor(0.0482)
Loss: tensor(0.0488)


In [21]:
# Report the accuracy, no need to change, but your should run the cell.
print('Accuracy train:', evaluate(net, train_dl, input_converter))
print('Accuracy test:', evaluate(net, test_dl, input_converter))

Accuracy train: 0.988
Accuracy test: 0.976


**Question 4**:  In the code cell below, create a CNN, called Q4Net, that takes the digit images as input and classifies them as one of the digits from 0-9. The specification of your network are as follows:
* The network should have, in addition to the input and output layers, two convolutional layers, each consisting of: a convolutional, rectified liner unit, and a pooling layer, in that order.
* The former convolutional layer has 8 filters (channels) and uses average-pooling, whereas the latter has 4 filters and uses max pooling.
* Both the convolutional layers use a kernel of size 3 x 3, a stride of 1, and no padding. 
* You can choose the type (average or max) and kernel and stride sizes of the pooling-layers, as long as the input you finally feed into the output layer is of the proper width as specified (see below).
* Following the convolutional layers there is a liner output and an activation layer (matching the task and data, as before). The with of the (input into the) output layer should be 100. 

As in question 2, train and evaluate your network using the train and evaluate functions (unmodified), and report on the accurarcy on both the training and test set. 

In [60]:
from torchsummary import summary
# Create a CNN network model

class Q4Net(nn.Module):
  
  def __init__(self, input_size, output_size): 
    super().__init__()
    
    self.layer_1 = nn.Sequential(
        # 28 x 28 x 1
        nn.Conv2d(1, 8, kernel_size=(3,3), stride=1, padding=0),  # 28 - 3 + 1 = 26;  26 x 26 
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)  # 13 x 13
    )

    self.layer_2 = nn.Sequential(
        # 1 x 12 x 12
        nn.Conv2d(8, 4, kernel_size=3, stride=1, padding=0),  # 13-3+1 = 11 ; 11 X 11
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2) #5 X 5
    )    
    self.layer_3 = nn.Flatten()
    self.layer_4 = nn.Linear(4*5*5, 8)   # My output is indeed 100
    return 

  def forward(self, x):
    x = self.layer_1(x)
    x = self.layer_2(x)
    x = self.layer_3(x) # Flatten
    x = self.layer_4(x)
    return nn.functional.log_softmax(x, dim=1)


net = Q4Net(MNIST_DIM*MNIST_DIM, MNIST_NUM_CLASSES)
print(net)
summary(net,(1,28,28))

# Train and evaluate your network.


Q4Net(
  (layer_1): Sequential(
    (0): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer_2): Sequential(
    (0): Conv2d(8, 4, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer_3): Flatten(start_dim=1, end_dim=-1)
  (layer_4): Linear(in_features=100, out_features=8, bias=True)
)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              80
              ReLU-2            [-1, 8, 26, 26]               0
         MaxPool2d-3            [-1, 8, 13, 13]               0
            Conv2d-4            [-1, 4, 11, 11]             292
              ReLU-5            [-1, 4, 11, 11]               0
         MaxPool2d-6              [-1, 4, 5, 5]             

In [71]:
# Train my neural network.
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
for epocs in range(5):
  for data in train_dl:
    X, y = data
    optimizer.zero_grad() # clear gradient information.
    output = net(X)
    loss = nn.functional.nll_loss(output, y)
    loss.backward() # do pack-propagation step
    optimizer.step() # tell optimizer that you finished batch/iteration.
  print(loss.data)

tensor(0.6454)
tensor(0.6007)
tensor(0.5942)
tensor(0.5352)
tensor(0.6315)


In [73]:
# Evaluate the trained network.
total = 0
correct = 0
with torch.no_grad():   # No need for keepnig track of necessary changes to the gradient.
  for data in test_dl:
    X, y = data
    output = net(X)
    for idx, val in enumerate(output):
      if torch.argmax(val) == y[idx]:
        correct += 1
      total += 1
  print('Accuracy:', round(correct/total, 3))

Accuracy: 0.77


**Question 5**:  In the code-cell below, using the provided data, write code for predicting from the dependent variables (first two columns) whether the dependent variable (last column) is 0 or 1. Your code should do the following tasks:

*   Convert the data to a tensor, and split it up into a training and test set(80% or records in the training set).
*   Using data loader, using mini-batch size *50*, feed the data it into a small fully-connected neural network with exactly two hidden layers. You can otherwise choose the specifications of your network, as long as the input and output layers are proper.
* Train the network using Adam as an optimizer.
* Report on the *accuracy* on both the test and the training set. Note, you will most likely need to write your own code for computing the accuracy (note, that there is no need for you to spend must time improving upon the accuracy, as long as you are somewhat confident you are doing things right).



In [26]:
# Question 5: reading in data. No need to change, but you must run the cell.
def read_data(name, records):
    with open(name) as f:
        for line in f:
            fields = line.split(',')
            value = 1 if fields[2].strip() == 'True' else 0
            record = (float(fields[0].strip()),
                      float(fields[1].strip()),
                      int(value))
            records.append(record)

# Data will be a Python list of tuples.
data = []
read_data('data.txt', data)
print(len(data))
print(data[0])
print(data[len(data)-1])


499
(0.07, -0.04, 0)
(-0.69, 0.69, 1)


In [29]:
# Question 5: your code. 
# First we create the Dataset
import pandas as pd

df = pd.DataFrame(data, columns=['number_1', 'number_2', 'Binary'])
print(df)

     number_1  number_2  Binary
0        0.07     -0.04       0
1        0.28      0.27       1
2        0.02      0.18       0
3        0.24     -0.22       1
4       -0.19     -0.42       1
..        ...       ...     ...
494     -0.08      0.57       1
495     -0.20      0.36       1
496     -0.28      0.62       1
497     -0.59      0.78       1
498     -0.69      0.69       1

[499 rows x 3 columns]


In [30]:
# Split the data into test and training sets.
# Now we split the DATA
from sklearn.model_selection import train_test_split
columns = df.columns.values.tolist()
independent = columns[:-1]
dependent = columns[-1]
train_X, test_X, train_y, test_y = train_test_split(df[independent], df[dependent], test_size=0.20)

In [52]:
import torch
BATCH_SIZE = 50
X = torch.tensor(train_X.values, dtype=torch.float)
y = torch.tensor(train_y.values, dtype=torch.int64)
train_ds = torch.utils.data.TensorDataset(X, y)
X = torch.tensor(test_X.values, dtype=torch.float)
y = torch.tensor(test_y.values, dtype=torch.int64)
test_ds = torch.utils.data.TensorDataset(X, y)

train_dl = torch.utils.data.DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)

In [65]:
# Create Neural Network Model (ANN)
# Import necessary libraries
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms

class MyANN(nn.Module):
  
  def __init__(self):
    super().__init__()
    self.fc1 = nn.Linear(2, 50) 
    self.av1 = nn.ReLU()   
    self.fc2 = nn.Linear(50, 50)    
    self.av2 = nn.ReLU()
    self.fc3 = nn.Linear(50, 50) 
    self.av3 = nn.ReLU() 
    self.fc4 = nn.Linear(50, 2) 
    self.av4 = nn.Sigmoid() 
    return


  def forward(self, x):
    x = self.fc1(x)
    x = self.av1(x)
    x = self.fc2(x)
    x = self.av2(x)
    x = self.fc3(x)
    x = self.av3(x)

    x = self.fc4(x)
    x = self.av4(x)
    return nn.functional.log_softmax(x, dim=1)

net = MyANN()
print(net)

MyANN(
  (fc1): Linear(in_features=2, out_features=50, bias=True)
  (av1): ReLU()
  (fc2): Linear(in_features=50, out_features=50, bias=True)
  (av2): ReLU()
  (fc3): Linear(in_features=50, out_features=50, bias=True)
  (av3): ReLU()
  (fc4): Linear(in_features=50, out_features=2, bias=True)
  (av4): Sigmoid()
)


In [67]:
# Train my neural network.
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
for epocs in range(10):
  for data in train_dl:
    X, y = data
    optimizer.zero_grad() # clear gradient information.
    output = net(X.view(-1, 2))
    loss = nn.functional.nll_loss(output, y)
    loss.backward() # do back-propagation step
  optimizer.step() # tell optimizer that you finished batch/iteration.
  print(loss.data)

tensor(0.6989)
tensor(0.6954)
tensor(0.6933)
tensor(0.6916)
tensor(0.6911)
tensor(0.6858)
tensor(0.6831)
tensor(0.6816)
tensor(0.6872)
tensor(0.6725)


In [69]:
# Evaluate the trained network.
total = 0
correct = 0
with torch.no_grad(): 
  for data in test_dl:
    X, y = data
    output = net(X.view(-1, 2))    
    for idx, val in enumerate(output):
      if torch.argmax(val) == y[idx]:
        correct += 1
      total += 1
  print('Accuracy:', round(correct/total, 3))

Accuracy: 0.77
